In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import spearmanr
from scipy.stats import pearsonr

In [ ]:
df = pd.read_csv('./data/Melbourne_housing_FULL.txt')
# df.head(50)

### Dataset columns:

- **suburb** (Suburb): The suburb where the property is located
- **address** (Address): The street address of the property
- **rooms** (Rooms): The number of rooms in the property
- **property_type** (Type): The type of the property (house, townhouse, unit, etc.)
- **price** (Price): The price at which the property was sold
- **sale_type** (Method): The method of sale (auction, private treaty, etc.)
- **seller** (SellerG): The agency or agent who sold the property
- **date** (Date): The date on which the property was sold
- **distance** (Distance): The distance of the property from Melbourne's central business district (CBD) in kilometers
- **postcode** (Postcode): The postcode of the suburb where the property is located
- **bedrooms** (Bedroom2): The number of bedrooms in the property (other than the master bedroom)
- **bathroom** (Bathroom): The number of bathrooms in the property
- **car_spaces** (Car): The number of car spaces in the property
- **land_size** (Landsize): The size of the land on which the property is located in square meters
- **building_area** (BuildingArea): The size of the building on the land in square meters
- **building_year** (YearBuilt): The year in which the building was constructed
- **council** (CouncilArea): The local government area in which the property is located
- **latitude** (Latitude): The latitude coordinate of the property
- **longitude** (Longitude): The longitude coordinate of the property
- **region** (Regionname): The general region in Melbourne (west, north, south-east, etc.)
- **property_count** (Propertycount): The number of properties in the suburb


# **Data Cleaning**

In [ ]:
print('\nDF basic info:')
print(df.info())

print('\nSummary stats:')
print(df.describe())

print('\nNull values:')
print(df.isnull().sum())


In [ ]:
# change column names
df.columns = [
    'suburb', 'address', 'rooms', 'property_type', 'price', 'sale_type', 'seller', 
    'date', 'distance', 'postcode', 'bedrooms', 'bathroom', 'car_spaces', 
    'land_size', 'building_area', 'building_year', 'council', 'latitude', 
    'longitude', 'region', 'property_count'
]

# print(df.columns)

In [ ]:
# value types
# print(df.dtypes)

In [ ]:
# identify columns that should be integer type
numerical_columns = df.select_dtypes(include=['float64']).columns

# columns to integers (remove .0) where appropriate
for col in numerical_columns:
    if df[col].dropna().apply(float.is_integer).all():
        df[col] = df[col].astype('Int64')

print(df.dtypes)

In [ ]:
unique_values = df.apply(lambda x: x.unique())

unique_values_table = pd.DataFrame(unique_values, columns=['Unique Values'])
print(unique_values_table)


In [ ]:
# change property type values
property_types = {
    'h': 'House',
    'u': 'Unit',
    't': 'Townhouse'
}

df['property_type'] = df['property_type'].replace(property_types)

In [ ]:
# group by 'suburb' and check the uniqueness of 'region' and 'council'
inconsistent_suburbs = df.groupby('suburb').agg(
    region_unique=('region', 'nunique'),
    council_area_unique=('council', 'nunique')  # Fixed the typo here
)

inconsistent_suburbs = inconsistent_suburbs[
    (inconsistent_suburbs['region_unique'] > 1) | (inconsistent_suburbs['council_area_unique'] > 1)
]

if not inconsistent_suburbs.empty:
    print('Inconsistent Suburbs where region or council does not match:')
    print(inconsistent_suburbs.index)
else:
    print("All values in 'suburb', 'region', and 'council' match correctly.")


In [ ]:
# replace missing 'region' and 'council' values by using the most frequent value within the same 'suburb'
df['region'] = df.groupby('suburb')['region'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))
df['council'] = df.groupby('suburb')['council'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))

print(df[['region', 'council']].isna().sum())

In [ ]:
land_size_median = df['land_size'].median()

# replace missing values with the median
df['land_size'] = df['land_size'].fillna(land_size_median)

print(f'Missing values in land_size after filling: {df['land_size'].isna().sum()}')

In [ ]:
# df.head()

In [ ]:
empty_values_count = df.isna().sum()

# show only columns with NaN values
empty_columns = empty_values_count[empty_values_count > 0]

print('Columns with empty values and their count:')
print(empty_columns)

In [ ]:
# drop all rows with empty (NaN) values
clean_df = df.dropna()

# print(clean_df)


# **Assessment Code**

## 1. Dropping Null Values and Casting Data Types
- Cast the 'price' column in the dataset to an integer type and remove any null values in the 'price' column. 
- Which of the following statements is true regarding dropping null values?

In [ ]:
# cast 'price' column to integer type, remove null values
df.loc[:, 'price'] = pd.to_numeric(df['price'], errors='coerce')
initial_rows = df.shape[0]
df = df.dropna(subset=['price'])
final_rows = df.shape[0]
rows_deleted = initial_rows - final_rows
 
print(f'Number of rows deleted: {rows_deleted}')
print(f'Remaining rows: {final_rows}')
print(df[['price']].isna().sum())


In [ ]:
# cast 'price' column to integer type, remove null values
clean_df.loc[:, 'price'] = pd.to_numeric(clean_df['price'], errors='coerce')
initial_rows = clean_df.shape[0]
clean_df = clean_df.dropna(subset=['price'])
final_rows = clean_df.shape[0]
rows_deleted = initial_rows - final_rows

print(f'Number of rows deleted: {rows_deleted}')
print(f'Remaining rows: {final_rows}')
print(clean_df[['price']].isna().sum())


### Answer
- **A:** dropping all null values will lead to a loss of 74% of the data
- **Correct answer:** dropping null values will lead to a loss of 21% of the data

## 2. Most Common Property Type and Its Percentage
What is the most common type of property, and what percentage of the properties does it represent?

In [ ]:
type_counts = df['property_type'].value_counts()

type_percentage = (type_counts / len(df)) * 100

# percentage with two decimal places and the percentage sign
type_percentage = type_percentage.apply(lambda x: f'{x:.2f}%')

print(type_percentage)


In [ ]:
type_counts = clean_df['property_type'].value_counts()

type_percentage = (type_counts / len(clean_df)) * 100

# percentage with two decimal places and the percentage sign
type_percentage = type_percentage.apply(lambda x: f'{x:.2f}%')

print(type_percentage)


### Answer
**A:** The most common type of property is houses, and they represent 67% of the properties.

## 3. Choosing the Right Graph Technique
- Create a graph to show the frequency of each property type n descending order. 
- Which graph technique is the most suitable and straightforward approach for this task?

In [ ]:
type_counts.sort_values(ascending=False).plot(kind='bar', color='lightgreen')

plt.title('Property Type Frequencies')
plt.xticks(rotation=0)
plt.show()

### Answer
- **A:** Barplot
- **Correct answer:** Countplot

## 4. Room Count vs Median Property Price
How does the number of rooms impact the median property price? 
- Represent this relationship using a bar chart.

In [ ]:
# group by 'rooms' and calculate the median 'price' for each group
median_price_per_room = df.groupby('rooms')['price'].median()

median_price_per_room = median_price_per_room.astype(int)

display(median_price_per_room)


In [ ]:
# group by 'rooms' and calculate the median 'price' for each group
median_price_per_room2 = clean_df.groupby('rooms')['price'].median()

median_price_per_room2 = median_price_per_room2.astype(int)

display(median_price_per_room2)


In [ ]:
median_price_per_room.plot(kind='bar', color='coral')
plt.title('Median Property Price by Number of Rooms')
plt.xlabel('Number of Rooms')
plt.ylabel('Median Property Price')
plt.xticks(rotation=0)
plt.show()

### Answer
**A:** The median property price generally increases with the number of rooms, but there are some exceptions where the price decreases or remains the same.

## 5. Relationship Between Property Price and Number of Rooms
Since the relationship between property price and number of rooms may not be linear, it is appropriate to use a non-parametric correlation coefficient such as Spearman's to determine the strength of the relationship. 
- Calculate the Spearman correlation coefficient to analyze the relationship between price and number of rooms.

In [ ]:
# calculate Spearman's rank correlation coefficient
spearman_corr, p_value = spearmanr(df['rooms'], df['price'])

print(f'Correlation coefficient: {spearman_corr}')
print(f'P_value: {p_value}')

In [ ]:
# calculate Spearman's rank correlation coefficient
spearman_corr2, p_value2 = spearmanr(clean_df['rooms'], clean_df['price'])

print(f'Correlation coefficient: {spearman_corr2}')
print(f'P_value: {p_value2}')

#### **Interpretation**

#### Spearman's Rank Correlation Coefficient: **0.5043**
The value of **0.5043** indicates a **moderate positive monotonic relationship** between the number of rooms and the property price. This suggests that, generally, as the number of rooms increases, the property price also tends to increase. However, the relationship is not perfectly linear, meaning there are variations and the correlation is moderate.

#### P-value: **0.0**
The **p-value of 0.0** is extremely small, indicating that the correlation is **statistically significant**. This means there is a very low probability that the observed correlation is due to random chance. Therefore, we can confidently conclude that the relationship between the number of rooms and property price is meaningful and not coincidental.

#### Conclusion:
The **moderate positive Spearman correlation (0.5043)** suggests that as the number of rooms increases, property prices tend to rise, although not in a perfectly linear manner. The **strong statistical significance (p-value = 0.0)** further supports the conclusion that this relationship is genuine and unlikely to be due to random variation.


### Answer
**A:** The correlation between property price and number of rooms is moderate.

## 6. CBD Distance and Property Price
Is there a relationship between the distance from the Central Business District (CBD) and the property price?
- Use a scatter plot to visualize the relationship and calculate a correlation coefficient to determine the strength and direction of the correlation.

In [ ]:
# fill empty values with mean
df['distance'] = df['distance'].fillna(df['distance'].mean())

In [ ]:
plt.scatter(df['distance'], df['price'], color='limegreen')
plt.title('Property Price vs Distance from CBD')
plt.xlabel('distance')
plt.ylabel('Property Price')
plt.grid(True)
plt.show()

# Pearson correlation coefficient
pearson_corr, p_value = pearsonr(df['distance'], df['price'])

print(f'Pearson correlation coefficient: {pearson_corr}')
print(f'P-value: {p_value}')

In [ ]:
plt.scatter(clean_df['distance'], clean_df['price'], color='limegreen')
plt.title('Property Price vs Distance from CBD')
plt.xlabel('distance')
plt.ylabel('Property Price')
plt.grid(True)
plt.show()

# Pearson correlation coefficient
pearson_corr2, p_value2 = pearsonr(clean_df['distance'], clean_df['price'])

print(f'Pearson correlation coefficient: {pearson_corr2}')
print(f'P-value: {p_value2}')

### Answer
**A:** There is a weak negative correlation between the distance from CBD and the property price.

## 7. Property Size vs Property Price
Is there a relationship between the property size and the property price?

In [ ]:
# Scatter plot to visualize the relationship
plt.scatter(df['land_size'], df['price'], color='hotpink')
plt.title('Property Price vs Property Size')
plt.xlabel('Size (land and building)')
plt.ylabel('price')
plt.grid(True)
plt.show()

# Pearson correlation coefficient
pearson_corr, p_value = pearsonr(df['land_size'], df['price'])

print(f'Pearson correlation coefficient: {pearson_corr}')
print(f'P-value: {p_value}')

In [ ]:
# Scatter plot to visualize the relationship
plt.scatter(clean_df['land_size'], clean_df['price'], color='hotpink')
plt.title('Property Price vs Property Size')
plt.xlabel('Size (land and building)')
plt.ylabel('price')
plt.grid(True)
plt.show()

# Pearson correlation coefficient
pearson_corr2, p_value2 = pearsonr(clean_df['land_size'], clean_df['price'])

print(f'Pearson correlation coefficient: {pearson_corr2}')
print(f'P-value: {p_value2}')

- **A:** There is a weak positive linear relationship between property size and property price.
- **Correct answer:** There is a weak positive correlation between the two variables, but the relationship might not be linear.

## 8. Land Size and Property Price: Quantile-Based Discretization and Bar Plot Comparison
When looking visually at the relationship between land size and property price, a scatter plot might not be the most informative way to visualize this relationship.
- Use a **quantile-based discretization function** to categorize the Landsize variable and create a bar plot.
- When using **q=7**, how does the relationship between land size and median property price change compared to using a lower quantile value?

When using a quantile-based discretization function to categorize the Landsize variable into quantiles (such as q=7), you're dividing the land size into 7 equal-sized groups, each containing approximately the same number of data points. This approach transforms a continuous variable (land size) into a categorical one, where each group corresponds to a range of land sizes. By comparing this method to a lower quantile value (such as q=4), you can observe how the relationship between land size and median property price changes.

#### Summary:
- Using **q=7** provides a higher level of detail, allowing you to more clearly observe subtle differences in property price across land sizes.
- Using a lower quantile value (e.g., **q=4**) smooths out these differences, resulting in a less granular view of the relationship, which could mask finer distinctions in property prices as they relate to land size.


### Answer
- **A:** The relationship between land size and property price becomes more pronounced when using q=7 compared to a lower quantile value.
- **Correct answer:** The frequency of properties with a larger land size having a lower price than properties with a smaller land size increases when using q=7, compared to a lower quantile value.

#### Explanation:
Using **q=7** means dividing the land size into 7 quantiles, which creates more granular groups, allowing for finer distinctions in the relationship between land size and property price. This increases the resolution of the data, helping to reveal more subtle trends and variations.

In contrast, using a lower quantile value (e.g., **q=4**) results in fewer groups, which may smooth out the relationship and make it harder to see more pronounced differences.

Thus, when using **q=7**, the relationship between land size and property price is more pronounced due to the finer categorization, making it easier to observe variations across different land size ranges.


## 9. Identifying Outliers in Property Prices: Box Plot and Tukey's Method
Does the dataset contain any properties that are priced significantly above or below the average? 
- Create a box plot to detect potential outliers, and u
- Use Tukey's method with a 1.5 threshold to create a new dataframe without these outliers. 
- What percentage of properties are identified as outliers using Tukey's method?

In [ ]:
# box plot to visually identify potential outliers
plt.figure(figsize=(10, 6))
plt.boxplot(df['price'])
plt.title('Property Prices')
plt.ylabel('price')
plt.show()

In [ ]:
# thresholds
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Tukey's method to detect outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# identify outliers
outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]

print(f'Lower bound: {lower_bound}')
print(f'Upper bound: {upper_bound}')

In [ ]:
# new df without outliers
outlier_free_df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

# percentage of outliers
percentage_outliers = (len(outliers) / len(df)) * 100

print(f'Percentage of outliers: {percentage_outliers:.2f}%')

### Answer
**A:** 4.69%

## 10.  Property Price Distribution: QQ Plot, Skewness, and Kurtosis Calculation
How is the property price distribution? 
- Create a QQ (Quantile-Quantile) Plot to compare the distribution of property prices to a normal distribution visually
- Calculate the skew and curtosis to get a better understanding of the shape of the distribution analytically.

In [ ]:
# create QQ plot to compare distribution
plt.figure(figsize=(8, 6))
stats.probplot(df['price'], dist='norm', plot=plt)
plt.title('QQ Plot: Property Prices vs Normal Distribution')
plt.show()

In [ ]:
# calculate skewness and kurtosis
skewness = df['price'].skew()
kurtosis = df['price'].kurtosis()

print(f'Skewness: {skewness:.2f}')
print(f'Kurtosis: {kurtosis:.2f}')

### Answer
**A:** The property price distribution is positively skewed.

#### Explanation:
**Skewness of 2.59** indicates a **positive skew**, meaning the distribution is **right-skewed** (with a long tail on the right side), where most property prices are lower, but a few high-priced properties create a longer tail on the right.

The **kurtosis value of 13.10** confirms the presence of **heavy tails** (more extreme high values than normal), which further supports the idea of **outliers** in the higher price range, a characteristic of a positively skewed distribution.


## 11. Property Price Distribution: Kolmogorov-Smirnov Test
What is the outcome of performing a normality test, such as the *Kolmogorov-Smirnov Test*, on the distribution of property prices? Specifically, what possible conclusions can be drawn from the test results regarding the similarity of the distribution to a normal distribution?

**Kolmogorov-Smirnov Test:**
- Null Hypothesis (H₀): The data follows a normal distribution.
- Alternative Hypothesis (H₁): The data does not follow a normal distribution.

In [ ]:
# standardize prices (mean 0, std 1)
standardized_prices = (df['price'] - df['price'].mean()) / df['price'].std()
# print(standardized_prices)

# Kolmogorov-Smirnov test
ks_statistic, p_value = stats.kstest(standardized_prices, 'norm')

print(f'KS statistic: {ks_statistic:.4f}')
print(f'p-value: {p_value:.4f}')

# interpretation based on p-value
if p_value < 0.05:
    print('The null hypothesis is rejected: The distribution of property prices is significantly different from a normal distribution.')
else:
    print('The null hypothesis cannot be rejected: The distribution of property prices is not significantly different from a normal distribution.')

### Answer
**A:** The test results indicate that the distribution of property prices in the Melbourne housing market dataset is significantly different from a normal distribution.

## 12. Property Prices Across Different Property Types: Box Plot Visualization
How do property prices differ between different property types? 
- Create a box plot to show the distribution of property prices for each property type.

In [ ]:
# box plot to show the distribution of property prices for each property type
plt.figure(figsize=(12, 8))
sns.boxplot(x='property_type', y='price', data=df)

plt.title('Property Price Distribution by Property Type', fontsize=16)
plt.xlabel('Property Type', fontsize=14)
plt.ylabel('Property Price', fontsize=14)

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

### Answer
**A:** Houses have higher property prices than units and townhouses.

## 13. Median Property Prices by Region: Identifying the Highest and Lowest Median Prices
- What is the region with the highest median price? 
- Conversely, which region has the lowest median price?

In [ ]:
# median of 'price' by 'region'
region_medians = df.groupby('region')['price'].median()

highest_median_region = region_medians.idxmax()
highest_median_price = int(region_medians.max())

lowest_median_region = region_medians.idxmin()
lowest_median_price = int(region_medians.min())

print(f'Region with the highest median price: {highest_median_region} (Median Price: {highest_median_price})')
print(f'Region with the lowest median price: {lowest_median_region} (Median Price: {lowest_median_price})')


### Answer
**A:** -Most expensive: Southern Metropolitan, Cheapest: Western Victoria

## 14. Region and Property Price: Chi-Square Test and Cramer's V Coefficient Analysis
Is there a statistically significant association between the region and the price of properties? 
- Use the *chi square test* to assess the independence between region and price
- Use *Cramer's V coefficient* to quantify the strength of the association between these two features
- Divide the price variable into five distinc ranges.
 
*It's worth noting that in order to perform this analysis, both features need to be categorical.*
*Use Cohen (1988) interpretation on Cramer's V, which depends on the degrees of freedom.*

In [ ]:
# categorize the 'price' column into five distinct ranges
price_bins = [0, 300000, 500000, 700000, 900000, np.inf]
price_labels = ['0-300k', '300k-500k', '500k-700k', '700k-900k', '900k+']
df['price_category'] = pd.cut(df['price'], bins=price_bins, labels=price_labels)

# create a contingency table of 'price_category' vs 'region'
contingency_table = pd.crosstab(df['price_category'], df['region'])

# Chi-Square Test of Independence
chi2_stat, p_val, dof, expected = stats.chi2_contingency(contingency_table)

# Cramer's V coefficient
n = contingency_table.sum().sum()
min_dim = min(contingency_table.shape)
cramers_v = np.sqrt(chi2_stat / (n * min_dim))

print(f'Chi-Square Test Statistic: {chi2_stat:.4f}')
print(f'p-value: {p_val:.4f}')
print(f"Cramer's V: {cramers_v:.4f}")

# Chi-Square Test Interpretation
if p_val < 0.05:
    print('There is a statistically significant association between region and price (reject H0).')
else:
    print('There is no statistically significant association between region and price (fail to reject H0).')

# Cramer's V Interpretation based on Cohen (1988)
if cramers_v < 0.1:
    print('The association between region and price is weak.')
elif cramers_v < 0.3:
    print('The association between region and price is moderate.')
else:
    print('The association between region and price is strong.')

# contingency table
sns.heatmap(contingency_table, annot=True, cmap='Blues', fmt='d')
plt.title('Contingency Table Heatmap')
plt.xticks(rotation=55)

plt.show()

### Answer
**A:** Yes, there is a significant relationship between the region and the price of properties, and the relationship is medium.

## 15. Percentage Change in Median Property Price between the Earliest and Latest Year
 How much was the percentage increase or decrease in *median price* of properties sold between the earliest year and the latest year?

In [ ]:
# double check results

# 1. Split rows between years (found in the date column), drop the ones where there's no year information.

df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Handle invalid dates as NaT

df['year'] = df['date'].dt.year

# drop rows where 'year' is NaN
df = df.dropna(subset=['year'])

# year column to integer for grouping
df['year'] = df['year'].astype(int)

# group by year
years_dict = {year: group.reset_index(drop=True) for year, group in df.groupby('year')}

print(f'DataFrames split by years: {list(years_dict.keys())}')


# 2. check the median price for each year's df and return the result (dyear, number of sales (= number of rows), and the median price fore ach year

def calculate_yearly_median(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'], errors='coerce') # check date format
    dataframe['year'] = dataframe['date'].dt.year # extract year
    
    # drop rows where 'year' or 'price' is missing
    dataframe = dataframe.dropna(subset=['year', 'price'])
    
    # year to integer and price to numeric
    dataframe['year'] = dataframe['year'].astype(int)
    dataframe['price'] = pd.to_numeric(dataframe['price'], errors='coerce')
    
    # group by year, calculate sales and median price
    result = dataframe.groupby('year').agg(
        number_of_sales=('price', 'count'),
        median_price=('price', 'median')
    ).reset_index()
    
    return result

yearly_stats = calculate_yearly_median(df)
print(yearly_stats)

print('')

# 3.  Calculate the percentage difference in median prices between first and last year
def calculate_difference(yearly_stats, year1, year2):
    price1 = yearly_stats.loc[yearly_stats['year'] == year1, 'median_price'].values[0]
    price2 = yearly_stats.loc[yearly_stats['year'] == year2, 'median_price'].values[0]
    
    percentage_diff = ((price2 - price1) / price1) * 100
    return percentage_diff

difference_2016_2018 = calculate_difference(yearly_stats, 2016, 2018)
print(f'The percentage difference between 2016 and 2018 in the median sales price is {difference_2016_2018:.2f}%')

### Answer
- **A:** -10.37%
- **Correct answer:** -5.48%

## 16. Identifying the Month with the Highest Property Sales Volume
Is there a specific month during which more houses are sold?

In [ ]:
# double-check sales per month again

df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

# Extract the month name from the Date
df['month'] = df['date'].dt.month_name()

# Group by month and count rows
rows_per_month = df['month'].value_counts().sort_index()

# results to df
result = result.sort_values(by='row_count', ascending=False)
result.columns = ['month', 'row_count']
print(result)


In [ ]:
# # Convert 'date' to datetime format
# df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Ensure Date is in datetime format

# # Extract the month from the 'date' column
# df['month'] = df['date'].dt.month_name()  # Get full month names (e.g., 'January', 'February')

# # Count the number of properties sold per month
# sales_per_month = df['month'].value_counts().sort_index()  # Count occurrences per month, sorted by month

# # Calculate the average sales per month
# average_sales_per_month = sales_per_month.mean()

# # Display the average sales per month
# print(f"Average sales per month: {average_sales_per_month:.2f}")

# # Month with the most sales
# most_sales_month = sales_per_month.idxmax()
# most_sales_count = sales_per_month.max()

# # Display the month with the most sales
# print(f"The month with the most houses sold is {most_sales_month} with {most_sales_count} sales.")


### Answer
- **A:** March.
- **Correct answer:** November.